In [1]:
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

0                                                                                       Gabrielle/
1                                                                                 Gabrielle/Orders
2                                                                            Gabrielle/Orders/AOIs
3                                          Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                               Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                                    ...                                           
249721    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
249722    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
249723    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
249724    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
249725    

In [3]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29735,Gabrielle/Shorelines/Merged/Auckland/Whangapoua_19FEB2023.shp
29762,Gabrielle/Shorelines/Merged/Auckland/Oneroa_27DEC2022.shp
29765,Gabrielle/Shorelines/Merged/Auckland/PakiriNorth_14OCT2021.shp
29769,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_23DEC2021.shp
29771,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_01MAR2023.shp
...,...
193514,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
193520,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
194627,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
194638,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_Wellington_13FEB2021.shp


In [4]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith((".jpg", ".jp2", ".tif", ".TIF"))]
images

33158     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
33159     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
33161       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
33164       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
33178       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
194864            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
194870            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
194873            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
194874            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
194877            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Name: 0, Length: 2702, dtype: ob

In [5]:
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith((".jpg", ".jp2", ".tif"))]

In [6]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    dirname = os.path.dirname(filename)
    dirname = os.path.dirname(dirname)
    RL_dirname = dirname.replace("MaxarImagery/HighFreq", "Retrolens")
    Maxar_dirname = dirname.replace("Retrolens", "MaxarImagery/HighFreq")
    all_files_in_folder = images[images.str.startswith((RL_dirname, Maxar_dirname))]
    if len(all_files_in_folder) == 0:
        return "", 0
    match, score, index = rapidfuzz.process.extractOne(query=filename, choices=all_files_in_folder, processor=fuzz_preprocess)
    return match, score

df["matched_image"], df["match_score"] = zip(*df.filename.apply(get_matching_image))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))
df[["filename", "matched_image", "match_score"]].sort_values(by="match_score").to_csv("shoreline_image_matching.csv", index=False)

Perfect matches: 2072
Imperfect matches: 698


### For those perfect matches, get the metadata from the corresponding matched image

In [7]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    image = rio.open(prefix + row.matched_image)
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read{row['filename']}")
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(thread_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

,filename,matched_image,match_score,n_lines,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,interleave,GCPs,res,CPS,compress,photometric
0,Gabrielle/Shorelines/Merged/Auckland/Whangapoua_19FEB2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Whangapoua/Whangapoua_19FEB2023.tif,100.0,3,GTiff,uint32,6.553600e+04,2804,6572,1,NaN,"| 0.60, 0.00, 1817329.44|\n| 0.00,-0.60, 6001088.43|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,5,"(0.5999999999999668, 0.6000000000000283)",True,NaN,NaN
1,Gabrielle/Shorelines/Merged/Auckland/Oneroa_27DEC2022.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Oneroa_27DEC2022.tif,100.0,3,GTiff,uint8,NaN,4482,3815,3,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""GRS 1980"",6378137,298.257222101,AUTHORITY[""EPSG"",""7019""]],AUTHORITY[""EP...","| 0.30, 0.00, 1779029.72|\n| 0.00,-0.30, 5928138.71|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,0,"(0.3000000000000208, 0.3)",True,NaN,NaN
2,Gabrielle/Shorelines/Merged/Auckland/PakiriNorth_14OCT2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Pakiri/PakiriNorth_14OCT2021.tif,100.0,14,GTiff,uint8,NaN,16238,19418,4,NaN,"| 0.50, 0.00, 1747718.26|\n| 0.00,-0.50, 5996962.31|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,4,"(0.5, 0.5)",True,NaN,NaN
3,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_23DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Tawharanui/Tawharanui_23DEC2021.tif,100.0,8,GTiff,uint8,NaN,7201,4822,4,NaN,"| 0.50, 0.00, 1762106.90|\n| 0.00,-0.50, 5975556.64|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,7,"(0.5, 0.5)",True,NaN,NaN
4,Gabrielle/Shorelines/Merged/Auckland/Tawharanui_01MAR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Tawharanui/Tawharanui_01MAR2023.tif,100.0,8,GTiff,uint8,NaN,12351,8084,3,NaN,"| 0.30, 0.00, 1761979.71|\n| 0.00,-0.30, 5975633.36|\n| 0.00, 0.00, 1.00|",128.0,128,True,pixel,7,"(0.30000000000000376, 0.300000000000023)",True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_17APR1952.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_17APR1952_mosaic.tif,100.0,5,GTiff,int32,2.147484e+09,14167,31280,1,NaN,"| 0.27, 0.00, 1763537.25|\n| 0.00,-0.27, 5469157.98|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,7,"(0.26838646488296874, 0.2683864648829636)",True,NaN,NaN
2035,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_02JAN1988_mosaic.jp2,100.0,9,JP2OpenJPEG,uint16,2.560000e+02,10654,18819,1,NaN,"| 0.39, 0.00, 1762602.58|\n| 0.00,-0.39, 5466346.26|\n| 0.00, 0.00, 1.00|",1024.0,1024,True,NaN,4,"(0.3894503758518079, 0.3894503758518002)",True,NaN,NaN
2036,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp,Retrolens/Wellington/KapitiSouth/Stack/KapitiSouth_06OCT1980_mosaic.tif,100.0,11,GTiff,int32,2.147484e+09,12620,29841,1,NaN,"| 0.36, 0.00, 1762518.60|\n| 0.00,-0.36, 5469658.24|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,10,"(0.35964806541597927, 0.3596480654159714)",True,NaN,NaN
2037,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp,Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif,100.0,6,GTiff,uint16,2.560000e+02,40393,28128,1,"PROJCS[""NZGD2000 / New Zealand Transverse Mercator 2000"",GEOGCS[""NZGD2000"",DATUM[""New_Zealand_Geodetic_Datum_2000"",SPHEROID[""GRS 1980"",6378137,298.257222101004,AUTHORITY[""EPSG"",""7019""]],AUTHORITY[...","| 0.13, 0.00, 1757756.38|\n| 0.00,-0.13, 5459482.04|\n| 0.00, 0.00, 1.00|",128.0,128,True,band,0,"(0.1284827509237925, 0.12848275092379952)",True,NaN,NaN


## Get Source for the failed matches

In [8]:
def get_source(filename):
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return "Unknown"
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            return "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            return "MAX"
        else:
            return "Unknown"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                return "MAX"
            else:
                return "Unknown"
        if len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
        return sources[0]

failures = df[df.match_score < 100]
failures["Source"] = process_map(get_source, failures.filename)
failures.Source.value_counts(dropna=False)

  0%|          | 0/698 [00:00<?, ?it/s]

Gabrielle/Shorelines/BayofPlenty/Papamoa/BOPLINZ_Papamoa_04APR2023.shp has ambiguous sources: ['LDS' None]
Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Taranaki/South Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp has ambiguous sources: ['LDS' None]
Retrolens/Otago/Moeraki_HampdenBeach/Stack/Shorelines/Moeraki_HampdenBeah_2006.shp has ambiguous sources: ['LDS' None]


/tmp/ipykernel_2041022/809631826.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  failures["Source"] = process_map(get_source, failures.filename)


Source
LDS        398
RL         118
MAX         70
Unknown     52
CRI         24
PLE         20
RLN          8
NEO          3
Max          2
JIL          1
SAT          1
RLS          1
Name: count, dtype: int64

In [9]:
failures[failures.Source != "LDS"].sort_values(by=["match_score", "filename"]).to_csv("failures.csv", index=False)

## Read LDS index tiles

In [10]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

4480      Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10041             Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11918          Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13888          Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14007               Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                                ...                                                       
230613             SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
230618        SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp
238223          

In [11]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/79 [00:00<?, ?it/s]

147407

## LDS index tile matching

In [12]:
LDS = failures[failures.Source == "LDS"].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source
112791,Retrolens/Waikato/Awakino/Shorelines/Awakino_15FEB2022.shp,,0.000000,LDS
110781,Retrolens/Waikato/WaikawauRiver/Shorelines/WaikawauRiver_10MAR2018.shp,,0.000000,LDS
113453,Retrolens/Waikato/UrawhitikiPoint/Shorelines/UrawhitikiPoint_10MAR2018.shp,,0.000000,LDS
113448,Retrolens/Waikato/UrawhitikiPoint/Shorelines/UrawhitikiPoint_15FEB2022.shp,,0.000000,LDS
113345,Retrolens/Waikato/KennedyBay/Shorelines/KennedyBay_16FEB2018.shp,,0.000000,LDS
...,...,...,...,...
161370,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS
128917,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_cliff.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Imagery/Stack/Oakura_03OCT2016.tif,96.551724,LDS
128869,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_beach.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Imagery/Stack/Oakura_03OCT2016.tif,96.551724,LDS
151566,Retrolens/Canterbury/GoreBay/Shorelines/GoreBay_8FEB2022.shp,Retrolens/Canterbury/GoreBay/Stack/GoreBay_LDS_18FEB2022.TIF,96.774194,LDS


In [13]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
LDS

  0%|          | 0/398 [00:00<?, ?it/s]

,filename,matched_index_tiles,date,DSASDate,matched_date,match_score,Pixel_ER
68,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",30.000000,0.3
197,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,33.750000,0.3
315,Retrolens/Otago/Moeraki_HampdenBeach/Stack/Shorelines/Moeraki_HampdenBeah_2006.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],1899-12-30,1899-12-30,02/12/17 to 08/02/18,36.000000,0.3
396,Retrolens/Canterbury/GoreBay/Shorelines/GoreBay_8FEB2022.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2022-02-08,08/02/2022,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",37.500000,0.3
168,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,38.095238,0.3
...,...,...,...,...,...,...,...
140,Gabrielle/Shorelines/Gisborne/Whareonga/Whareonga_16FEB2022.shp,"[Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photos-index-tiles-2021-2023Copy.shp, Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photo...",2022-02-16,16/02/2022,16/02/2022,100.000000,0.2
139,Retrolens/Taranaki/Kakaramea/Shorelines/Kakaramea_21NOV2017.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-11-21,21/11/2017,21/11/2017,100.000000,0.3
138,Gabrielle/Shorelines/Gisborne/WaipiraBay/WaipiraBay_08FEB2023.shp,"[Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photos-index-tiles-2021-2023Copy.shp, Gabrielle/Imagery/pre-storm/LINZ/Gisborne/Imagery/gisborne-02m-rural-aerial-photo...",2023-02-08,08/02/2023,08/02/2023,100.000000,0.2
136,Retrolens/Northland/WhareanaBay/Shorelines/WhareanaBay_07NOV2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Northland Footrpints/northland-04m-rural-aerial-photos-index-tiles-2014-2016.shp],2015-11-07,07/11/2015,07/11/2015,100.000000,0.4


In [14]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv("LDS_matches.csv", index=False)